In [ ]:
import sys
sys.path.append("/kaggle/input/readdata")
from torch.optim.lr_scheduler import StepLR
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
from tqdm import tqdm
from read_data import PCamDataset, transform

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),         
    transforms.RandomVerticalFlip(),          
    transforms.RandomRotation(degrees=15),     
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), 
])

In [ ]:
batch_size = 32
learning_rate = 1e-4
momentum = 0.9
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_pref = "/kaggle/input/train-data/"
val_pref = "/kaggle/input/val-data/"

In [ ]:

train_dataset = PCamDataset(train_pref+"camelyonpatch_level_2_split_train_x.h5", train_pref+"camelyonpatch_level_2_split_train_y.h5", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers = 4,pin_memory=True)
val_dataset = PCamDataset(val_pref+"camelyonpatch_level_2_split_valid_x.h5", val_pref+"camelyonpatch_level_2_split_valid_y.h5")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers = 4,pin_memory=True)


In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(vgg16.classifier[6].in_features, 2)
vgg16 = vgg16.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.SGD(vgg16.parameters(), lr=learning_rate, momentum=momentum)
scheduler = StepLR(optimizer_vgg, step_size=10, gamma=0.1)
history=[]

def evaluate(model, dataloader):
    model.eval()  
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy

def train_model(model, optimizer, train_loader, val_loader, epochs, model_name):
    for epoch in range(epochs):
        model.train()
        progress_bar = tqdm(train_loader, desc=f"{model_name} - Epoch {epoch+1}/{epochs}")

        for imgs, labels in progress_bar:
            imgs, labels = imgs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        train_loss, train_accuracy = evaluate(model,train_loader)
        val_loss, val_accuracy = evaluate(model,val_loader)
        history.append([train_loss, train_accuracy, val_loss, val_accuracy]) 
        scheduler.step()

In [ ]:
train_model(vgg16, optimizer_vgg, train_loader, val_loader, 25, "VGG16")
save_path = f"/kaggle/working/vgg.pth"
torch.save(vgg16.state_dict(), save_path)

print(history)